##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
%pip install ipykernel
%pip install pandas 
%pip install numpy
%pip install pyarrow
%pip install fastparquet
%pip install matplotlib
%pip install scikit-learn
%pip install imbalanced-learn
# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME 

## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [2]:
# Can have as many cells as you want for code
import pandas as pd
import matplotlib.pyplot as plt

# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file
filepath = "./data/catB_train.parquet" 

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

`clntnum`: Unique identifier for the client.

`race_desc`: Description of the client's race.

`ctrycode_desc`: Country code indicating the client's location.

`clttype`: Customer status.

`stat_flag`: Flag indicating ACTIVE, LAPSED or MATURED. E.g. if there’s at least one inforce policy, then the flag would be ACTIVE. If all of the client’s policies are all lapsed, then it is LAPSED.

`min_occ_date`: Date of the client's first interaction or policy purchase with the company.

`cltdob_fix`: Fixed or corrected date of birth of the client.

`cltsex_fix`: Fixed or corrected gender of the client.

In [38]:
###...code...###
test_df = pd.read_parquet(filepath)

# drop these columns
to_drop = []
for i in range(len(test_df.columns)):
    # if column has only 1 unique value, that column is unimportant as a feature
    if len(test_df[test_df.columns[i]].unique()) == 1:
        print(test_df.columns[i])
        # print(len(test_df[test_df.columns[i]].unique()))
        to_drop.append(i)

dropped_df = test_df.drop(test_df.columns[to_drop], axis=1)
dropped_df.head()

ape_gi_42e115
ape_ltc_1280bf
ape_inv_dcd836
ape_lh_d0adeb
ape_gi_a10d1b
ape_gi_29d435
ape_gi_856320
ape_gi_058815
ape_32c74c
sumins_gi_42e115
sumins_ltc_1280bf
sumins_inv_dcd836
sumins_lh_d0adeb
sumins_grp_22decf
sumins_gi_a10d1b
sumins_gi_29d435
sumins_lh_e22a6a
sumins_grp_e04c3a
sumins_gi_856320
sumins_grp_94baec
sumins_gi_058815
sumins_32c74c
prempaid_gi_42e115
prempaid_ltc_1280bf
prempaid_inv_dcd836
prempaid_lh_d0adeb
prempaid_gi_a10d1b
prempaid_gi_29d435
prempaid_gi_856320
prempaid_gi_058815
prempaid_32c74c
ape_d0adeb
ape_gi
f_hold_d0adeb
f_hold_gi
sumins_e22a6a
sumins_d0adeb
sumins_gi
prempaid_d0adeb
prempaid_gi
f_ever_bought_d0adeb
n_months_last_bought_d0adeb
f_ever_bought_ltc_1280bf
f_ever_bought_inv_dcd836
f_ever_bought_lh_d0adeb
f_ever_bought_32c74c
n_months_last_bought_ltc_1280bf
n_months_last_bought_inv_dcd836
n_months_last_bought_lh_d0adeb
giclaim_cnt_success
recency_giclaim_success
giclaim_cnt_unsuccess
recency_giclaim_unsuccess
flg_gi_claim_29d435_ever
flg_gi_claim_05881

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,min_occ_date,cltdob_fix,cltsex_fix,flg_substandard,flg_is_borderline_standard,...,recency_hlthclaim_success,hlthclaim_cnt_unsuccess,recency_hlthclaim_unsuccess,flg_hlthclaim_839f8a_ever,recency_hlthclaim_839f8a,flg_hlthclaim_14cb37_ever,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh
19550,91b546e924,Chinese,Singapore,P,ACTIVE,2017-10-31,1974-05-09,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4600,896bae548c,Chinese,Singapore,P,ACTIVE,2007-05-23,1979-11-11,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
13337,f364439ae6,Others,Singapore,P,ACTIVE,2019-08-31,1976-01-28,Male,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,2021-10-18,1976-03-19,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
19724,2647a81328,Chinese,Singapore,P,ACTIVE,2018-07-20,1995-07-31,Female,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [ ]:
# By Race
print(dropped_df.groupby('race_desc').count()['clntnum'])
print()

print(dropped_df.groupby('race_desc').agg({"f_purchase_lh": 'count'}))
print()

print(f'Chinese: {521/10520*100}')
print(f'Indian: {11/849 *100}')
print(f'Malay: {21/928 *100}')
print(f'Others: {112/1699 *100}')

In [ ]:
# By country
# only singaporeans are likely to buy insurance
dropped_df.groupby('ctrycode_desc').agg({"f_purchase_lh": 'count'}).sort_values(by='f_purchase_lh', ascending=False).iloc[:5]

In [ ]:
# By clttype
print(dropped_df.groupby('clttype').count()['clntnum'])
print()

print(dropped_df.groupby('clttype').agg({"f_purchase_lh": 'count'}))
print()

print(f'C type: {2/24*100}')
print(f'G type: {36/3311 *100}')
print(f'P type: {672/14657 *100}')


In [ ]:
# By stat_flag
print(dropped_df.groupby('stat_flag').count()['clntnum'])
print()

print(dropped_df.groupby('stat_flag').agg({"f_purchase_lh": 'count'}))
print()

print(f'Active: {694/17205 *100}')
print(f'Lapsed: {16/775 *100}')

In [ ]:
# by min_occ_year
new_df = dropped_df
new_df['min_occ_year'] = dropped_df['min_occ_date'].str.slice(stop=4) 

print(new_df.groupby('min_occ_year').count()['clntnum'])
print()

print(new_df.groupby('min_occ_year').agg({"f_purchase_lh": 'count'}))

In [ ]:
# by cltdob_fix
new_df['cltdob_fix_year'] = dropped_df['cltdob_fix'].str.slice(stop=4) 

# print(new_df.groupby('cltdob_fix_year').count()['clntnum'])
# print()

new_df.groupby('cltdob_fix_year').agg({"f_purchase_lh": 'count'}).sort_values('f_purchase_lh', ascending=False)


In [ ]:
# By Gender
print(dropped_df.groupby('cltsex_fix').count()['clntnum'])
print()

print(dropped_df.groupby('cltsex_fix').agg({"f_purchase_lh": 'count'}))
print()

print(f'Female: {345/8196 *100}')
print(f'Male: {363/9773 *100}')

### Further data transformations
Taking current date as 25-01-2024

In [39]:
# fill target na with 0s
dropped_df['f_purchase_lh'] = dropped_df['f_purchase_lh'].fillna(0)

In [40]:
dropped_df['f_purchase_lh'].sum()

710.0

In [41]:
# Convert datetime columns to duration/age
droprows = []
for row, client in dropped_df.iterrows():
    try:
        client['min_occ_date'] = pd.to_datetime(client['min_occ_date'])
        client['cltdob_fix'] = pd.to_datetime(client['cltdob_fix'])
    except:
        droprows.append(row)

dropped_df = dropped_df.drop(droprows,axis='index')
dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17960 entries, 19550 to 15795
Columns: 247 entries, clntnum to f_purchase_lh
dtypes: float64(44), int64(39), object(164)
memory usage: 34.0+ MB


In [42]:
curr_date = pd.to_datetime('2024-01-25')

dropped_df['min_occ_date'] = pd.to_datetime(dropped_df['min_occ_date'])
dropped_df['min_occ_year'] = dropped_df['min_occ_date'].dt.year
dropped_df['min_occ_month'] = dropped_df['min_occ_date'].dt.month
dropped_df['min_occ_days'] = (curr_date - dropped_df['min_occ_date']).dt.days

dropped_df['cltdob_fix'] = pd.to_datetime(dropped_df['cltdob_fix'])
dropped_df['age'] = ((curr_date - dropped_df['cltdob_fix']).dt.days/365.25).apply(lambda x:int(x))

dropped_df = dropped_df.drop(['min_occ_date', 'cltdob_fix'], axis=1)
dropped_df.head()

,clntnum,race_desc,ctrycode_desc,clttype,stat_flag,cltsex_fix,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,...,recency_hlthclaim_839f8a,flg_hlthclaim_14cb37_ever,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh,min_occ_year,min_occ_month,min_occ_days,age
19550,91b546e924,Chinese,Singapore,P,ACTIVE,Female,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,None,NaN,0.0,2017,10,2277,49
4600,896bae548c,Chinese,Singapore,P,ACTIVE,Male,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,None,NaN,0.0,2007,5,6091,44
13337,f364439ae6,Others,Singapore,P,ACTIVE,Male,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,None,NaN,0.0,2019,8,1608,47
15074,70f319cfe1,Chinese,Singapore,P,ACTIVE,Female,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,None,NaN,0.0,2021,10,829,47
19724,2647a81328,Chinese,Singapore,P,ACTIVE,Female,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,None,NaN,0.0,2018,7,2015,28


In [43]:
dropped_df = dropped_df.drop(['clntnum'], axis=1)

### Encoding categorical variables

#### Test for PCA relevance

In [8]:
temp = dropped_df[['race_desc','ctrycode_desc','stat_flag','cltsex_fix','f_purchase_lh']]
temp.info()
temp = temp.dropna(subset=['race_desc','ctrycode_desc','stat_flag','cltsex_fix'])
temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17960 entries, 19550 to 15795
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   race_desc      13995 non-null  object 
 1   ctrycode_desc  17940 non-null  object 
 2   stat_flag      17960 non-null  object 
 3   cltsex_fix     17958 non-null  object 
 4   f_purchase_lh  17960 non-null  float64
dtypes: float64(1), object(4)
memory usage: 841.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 13995 entries, 19550 to 15795
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   race_desc      13995 non-null  object 
 1   ctrycode_desc  13995 non-null  object 
 2   stat_flag      13995 non-null  object 
 3   cltsex_fix     13995 non-null  object 
 4   f_purchase_lh  13995 non-null  float64
dtypes: float64(1), object(4)
memory usage: 656.0+ KB


In [ ]:
round((dropped_df['f_purchase_lh'].sum() - temp['f_purchase_lh'].sum())/dropped_df['f_purchase_lh'].sum()*100,2)

In [9]:
cat_cols = list(temp.columns)[:-1]

In [10]:
cat_cols = list(temp.columns)[:-1]
temp_encoded = pd.get_dummies(temp, columns=cat_cols)
temp_encoded.info()

temp_encoded = temp_encoded.loc[:, temp_encoded.sum() > 1] # remove cols with only 1 entry i.e. only one obs in dataset
temp_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13995 entries, 19550 to 15795
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   f_purchase_lh                       13995 non-null  float64
 1   race_desc_Chinese                   13995 non-null  bool   
 2   race_desc_Indian                    13995 non-null  bool   
 3   race_desc_Malay                     13995 non-null  bool   
 4   race_desc_Others                    13995 non-null  bool   
 5   ctrycode_desc_Australia             13995 non-null  bool   
 6   ctrycode_desc_Bosnia-Herzegovina    13995 non-null  bool   
 7   ctrycode_desc_Brunei Darussalam     13995 non-null  bool   
 8   ctrycode_desc_Canada                13995 non-null  bool   
 9   ctrycode_desc_China                 13995 non-null  bool   
 10  ctrycode_desc_Denmark               13995 non-null  bool   
 11  ctrycode_desc_Hong Kong             13995 

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
temp_encoded_train = temp_encoded.drop(['f_purchase_lh'],axis=1)
pca.fit(temp_encoded_train)
x_pca = pca.transform(temp_encoded_train)

print(x_pca.shape)

print(temp_encoded_train.shape)

(13995, 2)
(13995, 26)


In [ ]:
plt.scatter(x_pca[:,0],x_pca[:,1], c=temp_encoded['f_purchase_lh'])

plt.xlabel('First Principal Component')

plt.ylabel('Second Principal Component')

In [ ]:
pca.components_

In [12]:
# Calculate PC 1/2 with significant variables (>0.05)
pc_1 = list(i if abs(i) >= 0.05 else 0 for i in pca.components_[0])
pc_2 = list(i if abs(i) >= 0.05 else 0 for i in pca.components_[1])

Assume significance > 0.05 <br>
1st PC significance: race_desc_Chinese, race_desc_Others, cltsex_fix_Female, cltsex_fix_Male <br>
2nd PC significance: race_desc_Chinese, race_desc_Indian, race_desc_Malay, race_desc_Others, cltsex_fix_Female, cltsex_fix_Male

### PCA Analysis - TBC

In [13]:
temp_encoded['pc1'] = temp_encoded['race_desc_Chinese']*pc_1[0] + temp_encoded['race_desc_Others']*pc_1[3] + temp_encoded['cltsex_fix_Female']*pc_1[-2] + temp_encoded['cltsex_fix_Male']*pc_1[-1]
temp_encoded['pc2'] = temp_encoded['race_desc_Chinese']*pc_2[0] + temp_encoded['race_desc_Indian']*pc_2[1] + temp_encoded['race_desc_Malay']*pc_2[2] + temp_encoded['race_desc_Others']*pc_2[3] + temp_encoded['cltsex_fix_Female']*pc_2[-2] + temp_encoded['cltsex_fix_Male']*pc_2[-1]

In [ ]:
scatter_plot = plt.scatter(temp_encoded['pc1'],temp_encoded['pc2'], c=temp_encoded['f_purchase_lh'])

handles, labels = scatter_plot.legend_elements()

# Create the legend using handles and labels
plt.legend(handles, labels)

plt.xlabel('First Principal Component')

plt.ylabel('Second Principal Component')

#plt.axvline(x=-0.635, color='red', linestyle='--', label='Vertical Line at x=-0.635')
#plt.axvline(x=-0.665, color='red', linestyle='--', label='Vertical Line at x=-0.665')

#plt.axhline(y=-0.63, color='red', linestyle='--', label='Vertical Line at x=-0.63')
#plt.axhline(y=-0.67, color='red', linestyle='--', label='Vertical Line at x=-0.67')

plt.show()

In [14]:
# add PC1-2 to dropped df
pcdf = dropped_df[['race_desc','ctrycode_desc','stat_flag','cltsex_fix','f_purchase_lh']]
pcdf_encoded = pd.get_dummies(pcdf, columns=cat_cols)

# pcdf_encoded.info()
pcdf_encoded['pc1'] = pcdf_encoded['race_desc_Chinese']*pc_1[0] + pcdf_encoded['race_desc_Others']*pc_1[3] + pcdf_encoded['cltsex_fix_Female']*pc_1[-2] + pcdf_encoded['cltsex_fix_Male']*pc_1[-1]
pcdf_encoded['pc2'] = pcdf_encoded['race_desc_Chinese']*pc_2[0] + pcdf_encoded['race_desc_Indian']*pc_2[1] + pcdf_encoded['race_desc_Malay']*pc_2[2] + pcdf_encoded['race_desc_Others']*pc_2[3] + pcdf_encoded['cltsex_fix_Female']*pc_2[-2] + pcdf_encoded['cltsex_fix_Male']*pc_2[-1]
pcdf_encoded.head()

,f_purchase_lh,race_desc_Chinese,race_desc_Indian,race_desc_Malay,race_desc_Others,ctrycode_desc_Australia,ctrycode_desc_Bosnia-Herzegovina,ctrycode_desc_Brunei Darussalam,ctrycode_desc_Canada,ctrycode_desc_China,...,ctrycode_desc_United Kingdom,ctrycode_desc_United States,ctrycode_desc_Unknown Country Code,stat_flag_ACTIVE,stat_flag_LAPSED,stat_flag_MATURED,cltsex_fix_Female,cltsex_fix_Male,pc1,pc2
19550,0.0,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,-0.797042,-0.753830
4600,0.0,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,0.608138,-0.912619
13337,0.0,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,True,0.756313,0.400466
15074,0.0,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,-0.797042,-0.753830
19724,0.0,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,-0.797042,-0.753830


In [15]:
dropped_df = pd.merge(dropped_df, pcdf_encoded[['pc1','pc2']], left_index=True, right_index=True)
dropped_df = dropped_df.drop(['clntnum','race_desc','ctrycode_desc','stat_flag','cltsex_fix'], axis=1)
dropped_df.head()

,clttype,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,flg_has_health_claim,flg_has_life_claim,flg_gi_claim,flg_is_proposal,flg_with_preauthorisation,...,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh,min_occ_year,min_occ_month,min_occ_days,age,pc1,pc2
19550,P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2017,10,2277,49,-0.797042,-0.753830
4600,P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2007,5,6091,44,0.608138,-0.912619
13337,P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2019,8,1608,47,0.756313,0.400466
15074,P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2021,10,829,47,-0.797042,-0.753830
19724,P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2018,7,2015,28,-0.797042,-0.753830


In [16]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[['P', 'G', 'C']])

# Fit and transform the ordinal_column
dropped_df['clttype'] = encoder.fit_transform(dropped_df[['clttype']])

dropped_df.head()

,clttype,flg_substandard,flg_is_borderline_standard,flg_is_revised_term,flg_is_rental_flat,flg_has_health_claim,flg_has_life_claim,flg_gi_claim,flg_is_proposal,flg_with_preauthorisation,...,recency_hlthclaim_14cb37,giclaim_amt,recency_giclaim,f_purchase_lh,min_occ_year,min_occ_month,min_occ_days,age,pc1,pc2
19550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2017,10,2277,49,-0.797042,-0.753830
4600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2007,5,6091,44,0.608138,-0.912619
13337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2019,8,1608,47,0.756313,0.400466
15074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2021,10,829,47,-0.797042,-0.753830
19724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,None,NaN,0.0,2018,7,2015,28,-0.797042,-0.753830


In [17]:
dropped_df = dropped_df.drop(['annual_income_est'], axis=1)
object_columns = dropped_df.select_dtypes(include='object').columns
print(object_columns)

Index(['hh_20', 'pop_20', 'hh_size_est', 'ape_grp_6fc3e6', 'ape_grp_de05ae',
       'ape_grp_945b5a', 'ape_grp_6a5788', 'ape_ltc_43b9d5', 'ape_grp_9cdedf',
       'ape_grp_1581d7',
       ...
       'n_months_last_bought_grp_70e1dd', 'n_months_last_bought_grp_e04c3a',
       'n_months_last_bought_grp_fe5fb8', 'n_months_last_bought_grp_94baec',
       'n_months_last_bought_grp_e91421', 'n_months_last_bought_lh_f852af',
       'n_months_last_bought_lh_947b15', 'n_months_last_bought_32c74c',
       'hlthclaim_amt', 'giclaim_amt'],
      dtype='object', length=155)


In [18]:
dropped_df = dropped_df.drop(['annual_income_est'], axis=1)
object_columns = dropped_df.select_dtypes(include='object').columns
for col in object_columns:
    try:
        dropped_df[col] = pd.to_numeric(dropped_df[col], errors='coerce')
        mode_value = dropped_df[col].mode().iloc[0]
        dropped_df[col].fillna(mode_value, inplace=True)
    except:
        print(col)

C:\Users\65979\AppData\Local\Temp\ipykernel_21172\1431553615.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dropped_df[col].fillna(mode_value, inplace=True)
C:\Users\65979\AppData\Local\Temp\ipykernel_21172\1431553615.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

C:\Users\65979\AppData\Local\Temp\ipykernel_21172\1431553615.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dropped_df[col].fillna(mode_value, inplace=True)
C:\Users\65979\AppData\Local\Temp\ipykernel_21172\1431553615.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [26]:
X = dropped_df.drop('f_purchase_lh', axis=1)
y = dropped_df['f_purchase_lh']

In [27]:
%%time
import numpy as np
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2, weights="uniform")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

CPU times: total: 2min 42s
Wall time: 2min 9s


In [33]:
class_counts = y_train.value_counts()
class_weights = class_counts.max() / class_counts
class_weights

f_purchase_lh
0.0     1.000000
1.0    24.795332
Name: count, dtype: float64

In [35]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calculate class weights to address class imbalance
class_counts = y_train.value_counts()
class_weights = class_counts.max() / class_counts

# Define XGBoost parameters and set the scale_pos_weight parameter
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'scale_pos_weight': class_weights[0] / class_weights[1],  # Adjust the scale_pos_weight based on the class imbalance
    'seed': 40,
}

# Convert the data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Train the XGBoost model
model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions on the test set
y_pred = model.predict(dtest)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluate the model
print(confusion_matrix(y_test, y_pred_binary))
print(classification_report(y_test, y_pred_binary))

[[3440    1]
 [ 150    1]]
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      3441
         1.0       0.50      0.01      0.01       151

    accuracy                           0.96      3592
   macro avg       0.73      0.50      0.50      3592
weighted avg       0.94      0.96      0.94      3592



In [ ]:
import numpy as np
import seaborn
%matplotlib inline

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [55]:
def testing_hidden_data(hidden_data: pd.DataFrame, y) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    
    to_drop = []
    for i in range(len(hidden_data.columns)):
        # if column has only 1 unique value, that column is unimportant as a feature
        if len(hidden_data[hidden_data.columns[i]].unique()) == 1:
            # print(hidden_data.columns[i])
            # print(len(test_df[test_df.columns[i]].unique()))
            to_drop.append(i)

    dropped_df = hidden_data.drop(hidden_data.columns[to_drop], axis=1)
    
    droprows = []
    for row, client in dropped_df.iterrows():
        try:
            client['min_occ_date'] = pd.to_datetime(client['min_occ_date'])
            client['cltdob_fix'] = pd.to_datetime(client['cltdob_fix'])
        except:
            droprows.append(row)

    dropped_df = dropped_df.drop(droprows,axis='index')
    # dropped_df.info()
    
    curr_date = pd.to_datetime('2024-01-25')

    dropped_df['min_occ_date'] = pd.to_datetime(dropped_df['min_occ_date'])
    dropped_df['min_occ_year'] = dropped_df['min_occ_date'].dt.year
    dropped_df['min_occ_month'] = dropped_df['min_occ_date'].dt.month
    dropped_df['min_occ_days'] = (curr_date - dropped_df['min_occ_date']).dt.days

    dropped_df['cltdob_fix'] = pd.to_datetime(dropped_df['cltdob_fix'])
    dropped_df['age'] = ((curr_date - dropped_df['cltdob_fix']).dt.days/365.25).apply(lambda x:int(x))

    dropped_df = dropped_df.drop(['min_occ_date', 'cltdob_fix'], axis=1)
    
    cat_cols = ['race_desc','ctrycode_desc','stat_flag','cltsex_fix']
    
    # add PC1-2 to dropped df
    pcdf = dropped_df[['race_desc','ctrycode_desc','stat_flag','cltsex_fix']]
    pcdf_encoded = pd.get_dummies(pcdf, columns=cat_cols)

    pcdf_encoded['pc1'] = pcdf_encoded['race_desc_Chinese']*pc_1[0] + pcdf_encoded['race_desc_Others']*pc_1[3] + pcdf_encoded['cltsex_fix_Female']*pc_1[-2] + pcdf_encoded['cltsex_fix_Male']*pc_1[-1]
    pcdf_encoded['pc2'] = pcdf_encoded['race_desc_Chinese']*pc_2[0] + pcdf_encoded['race_desc_Indian']*pc_2[1] + pcdf_encoded['race_desc_Malay']*pc_2[2] + pcdf_encoded['race_desc_Others']*pc_2[3] + pcdf_encoded['cltsex_fix_Female']*pc_2[-2] + pcdf_encoded['cltsex_fix_Male']*pc_2[-1]
    
    dropped_df = pd.merge(dropped_df, pcdf_encoded[['pc1','pc2']], left_index=True, right_index=True)
    dropped_df = dropped_df.drop(['clntnum','race_desc','ctrycode_desc','stat_flag','cltsex_fix'], axis=1)

    # from sklearn.preprocessing import OrdinalEncoder
    encoder = OrdinalEncoder(categories=[['P', 'G', 'C']])

    # Fit and transform the ordinal_column
    dropped_df['clttype'] = encoder.fit_transform(dropped_df[['clttype']])

    dropped_df = dropped_df.drop(['annual_income_est'], axis=1)
    object_columns = dropped_df.select_dtypes(include='object').columns
    for col in object_columns:
        try:
            dropped_df[col] = pd.to_numeric(dropped_df[col], errors='coerce')
            mode_value = dropped_df[col].mode().iloc[0]
            dropped_df[col].fillna(mode_value, inplace=True)
        except:
            print(col)
    X = dropped_df
    # import numpy as np
    # from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=2, weights="uniform")
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    dtest = xgb.DMatrix(X, label=y)

   
    # Make predictions on the test set
    y_pred = model.predict(dtest)
    y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

    # Evaluate the model
    print(confusion_matrix(y, y_pred_binary))
    print(classification_report(y, y_pred_binary))

    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [56]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
y = test_df['f_purchase_lh']
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df,y))

C:\Users\65979\AppData\Local\Temp\ipykernel_21172\2200510630.py:69: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dropped_df[col].fillna(mode_value, inplace=True)
C:\Users\65979\AppData\Local\Temp\ipykernel_21172\2200510630.py:69: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

XGBoostError: [18:09:50] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\data\data.cc:501: Check failed: this->labels.Size() % this->num_row_ == 0 (32 vs. 0) : Incorrect size for labels.

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!